In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow
from tensorflow import keras
# from tensorflowkeras.utils.vis_utils import plot_model
import random
from sklearn.model_selection import train_test_split
from scipy import ndimage
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Lambda,Dense,Flatten,Input

import tensorflow as tf

import time

2024-04-30 22:47:10.159397: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 22:47:10.162195: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 22:47:10.192478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 22:47:10.664620: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
inputs = keras.Input((32, 32, 3))
inputs

<KerasTensor shape=(None, 32, 32, 3), dtype=float32, sparse=None, name=keras_tensor>

In [29]:
class custom_MTL():
    def __init__(self) -> None:
        
        self.layer_list=[]
        x_shape, y_angle_shape, y_cifar10_shape=(50000, 32, 32, 3), (50000, 4), (50000, 10)
        angle_classifier_no=4
        cifar10_classifier_no=10
        inputs = keras.Input((32, 32, 3))
        conv_base = self.get_convbase(inputs)
        angle_classifier = self.get_classifier(conv_base, angle_classifier_no, "angle",count=15)
        cifar10_classifier = self.get_classifier(conv_base, cifar10_classifier_no, "cifar10",count=22)
        self.model = Model(
            inputs=inputs, 
            outputs=[cifar10_classifier, angle_classifier]
        )
        tf.keras.utils.plot_model(self.model, to_file='final_mtl_model.png', show_shapes=True,show_layer_names=True)
    
   
    def get_convbase(self,inputs):
    
    # reg = keras.regularizers.l2(1e-4)
    
    # initializer = keras.initializers.HeNormal()


        x = Conv2D(16, (3, 3), padding="same",name='1_conv2D_1')(inputs)
        x = Activation("relu",name='2_activation_1')(x)
        x = BatchNormalization(axis=-1,name='3_batch_norm_1')(x)
        x = MaxPooling2D(pool_size=(3, 3),name='4_maxPool_1')(x)
        x = Dropout(0.25,name='5_dropout_1')(x)
        
        x = Conv2D(32, (3, 3), padding="same",name='6_conv2D_2')(x)
        x = Activation("relu",name='7_activation_2')(x)
        x = BatchNormalization(axis=-1,name='8_batch_norm_2')(x)
        x = MaxPooling2D(pool_size=(2, 2),name='9_maxPool_2')(x)
        x = Dropout(0.25,name='10_dropout_2')(x)
        
        x = Conv2D(32, (3, 3), padding="same",name='11_conv2D_3')(x)
        x = Activation("relu",name='12_activation_3')(x)
        x = BatchNormalization(axis=-1,name='13_batch_norm_3')(x)
        x = MaxPooling2D(pool_size=(2, 2),name='14_maxPool_3')(x)
        x = Dropout(0.25,name='15_dropout_3')(x)
        
        return x
    
    def get_classifier(self,x, class_no, name,count):
    
        x = Flatten(name=f'{count+1}_layer')(x)
        x = Dense(128,name=f'{count+2}_layer')(x)
        x = Activation("relu",name=f'{count+3}_layer')(x)
        x = BatchNormalization(name=f'{count+4}_layer')(x)
        x = Dropout(0.5,name=f'{count+5}_layer')(x)
        x = Dense(class_no,name=f'{count+6}_layer')(x)
        x = Activation("softmax", name=name)(x)

        return x

    def load_weights(self):
        self.model.load_weights('./custom_mtl_model.h5')
        
    def execute_predict(self,input_data):
        st1=time.perf_counter()
        out=self.model.predict(input_data)
        et1=time.perf_counter()
        el1=et1-st1
        print(f'Elapsed Time: {el1}')
        return out
    
    def print_lays(self):
        for lay in self.model.layers:
            self.layer_list.append(lay)
            print(lay)
    def execute_lbl(self,input_data):
        st2=time.perf_counter()
        out=buffer=input_data
        for idx in range(1,len(self.model.layers)):
            print(f'Executing: {self.model.layers[idx]}')
            if idx <= 15:
                out=buffer=self.model.layers[idx](out)
            elif idx in [16,18,20,22,24,26,28]:
                out=self.model.layers[idx](out)
            elif idx in [17,19,21,23,25,27,29]:
                buffer=self.model.layers[idx](buffer)
        et2=time.perf_counter()
        el2=et2-st2
        print(f'Elapsed Time: {el2}')
        return out,buffer
        
    


In [30]:
image = np.random.rand(1,32,32,3)

In [31]:
obj=custom_MTL()
obj.load_weights()

# out=obj.model.predict(image)
obj.print_lays()

<InputLayer name=input_layer_7, built=True>
<Conv2D name=1_conv2D_1, built=True>
<Activation name=2_activation_1, built=True>
<BatchNormalization name=3_batch_norm_1, built=True>
<MaxPooling2D name=4_maxPool_1, built=True>
<Dropout name=5_dropout_1, built=True>
<Conv2D name=6_conv2D_2, built=True>
<Activation name=7_activation_2, built=True>
<BatchNormalization name=8_batch_norm_2, built=True>
<MaxPooling2D name=9_maxPool_2, built=True>
<Dropout name=10_dropout_2, built=True>
<Conv2D name=11_conv2D_3, built=True>
<Activation name=12_activation_3, built=True>
<BatchNormalization name=13_batch_norm_3, built=True>
<MaxPooling2D name=14_maxPool_3, built=True>
<Dropout name=15_dropout_3, built=True>
<Flatten name=23_layer, built=True>
<Flatten name=16_layer, built=True>
<Dense name=24_layer, built=True>
<Dense name=17_layer, built=True>
<Activation name=25_layer, built=True>
<Activation name=18_layer, built=True>
<BatchNormalization name=26_layer, built=True>
<BatchNormalization name=19_lay

In [32]:
obj.execute_predict(image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Elapsed Time: 0.09987428796011955


[array([[0.0797014 , 0.07793119, 0.07088789, 0.11130471, 0.10851418,
         0.10154222, 0.16401647, 0.08009152, 0.09813018, 0.10788022]],
       dtype=float32),
 array([[0.24713223, 0.20949385, 0.2326632 , 0.31071073]], dtype=float32)]

In [33]:
obj.execute_lbl(image)

Executing: <Conv2D name=1_conv2D_1, built=True>
Executing: <Activation name=2_activation_1, built=True>
Executing: <BatchNormalization name=3_batch_norm_1, built=True>
Executing: <MaxPooling2D name=4_maxPool_1, built=True>
Executing: <Dropout name=5_dropout_1, built=True>
Executing: <Conv2D name=6_conv2D_2, built=True>
Executing: <Activation name=7_activation_2, built=True>
Executing: <BatchNormalization name=8_batch_norm_2, built=True>
Executing: <MaxPooling2D name=9_maxPool_2, built=True>
Executing: <Dropout name=10_dropout_2, built=True>
Executing: <Conv2D name=11_conv2D_3, built=True>
Executing: <Activation name=12_activation_3, built=True>
Executing: <BatchNormalization name=13_batch_norm_3, built=True>
Executing: <MaxPooling2D name=14_maxPool_3, built=True>
Executing: <Dropout name=15_dropout_3, built=True>
Executing: <Flatten name=23_layer, built=True>
Executing: <Flatten name=16_layer, built=True>
Executing: <Dense name=24_layer, built=True>
Executing: <Dense name=17_layer, bui

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'np_array'